In [1]:
!pip install py2neo
!pip install wikipedia
!pip install spacy==3.0.3
!pip install scikit-learn
!pip install pandas

In [2]:
import json
import re
import urllib
from pprint import pprint
import time
from tqdm import tqdm

from py2neo import Node, Graph, Relationship, NodeMatcher
from py2neo.bulk import merge_nodes

import numpy as np
import pandas as pd
import wikipedia
from sklearn.metrics.pairwise import cosine_similarity

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span, Token

from collections import Counter

print(spacy.__version__)

3.0.3


In [3]:
!python3 -m spacy download en_core_web_md
# !python3 -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 MB 6.9 MB/s eta 0:00:0000:01m00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [4]:
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
VERBS = ['ROOT', 'advcl']
OBJECTS = ["dobj", "dative", "attr", "oprd", 'pobj']
ENTITY_LABELS = ['PERSON', 'NORP', 'GPE', 'ORG', 'FAC', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART']


non_nc = spacy.load('en_core_web_lg')

nlp = spacy.load('en_core_web_md')

nlp.add_pipe('merge_noun_chunks')



print(non_nc.pipe_names)
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']
['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer', 'merge_noun_chunks']


## Data Cleaning

In [5]:
def remove_special_characters(text):
    
    regex = re.compile(r'[\n\r\t]')
    clean_text = regex.sub(" ", text)
    
    return clean_text


def remove_stop_words_and_punct(text, print_text=False):
    
    result_ls = []
    rsw_doc = non_nc(text)
    
    for token in rsw_doc:
        if print_text:
            print(token, token.is_stop)
            print('--------------')
        if not token.is_stop and not token.is_punct:
            result_ls.append(str(token))
    
    result_str = ' '.join(result_ls)

    return result_str


def create_svo_lists(doc, print_lists):
    
    subject_ls = []
    verb_ls = []
    object_ls = []

    for token in doc:
        if token.dep_ in SUBJECTS:
            subject_ls.append((token.lower_, token.idx))
        elif token.dep_ in VERBS:
            verb_ls.append((token.lemma_, token.idx))
        elif token.dep_ in OBJECTS:
            object_ls.append((token.lower_, token.idx))

    if print_lists:
        print('SUBJECTS: ', subject_ls)
        print('VERBS: ', verb_ls)
        print('OBJECTS: ', object_ls)
    
    return subject_ls, verb_ls, object_ls


def remove_duplicates(tup, tup_posn):
    
    check_val = set()
    result = []
    
    for i in tup:
        if i[tup_posn] not in check_val:
            result.append(i)
            check_val.add(i[tup_posn])
            
    return result


def remove_dates(tup_ls):
    
    clean_tup_ls = []
    for entry in tup_ls:
        if not entry[2].isdigit():
            clean_tup_ls.append(entry)
    return clean_tup_ls


def create_svo_triples(text, print_lists=False):
    
    clean_text = remove_special_characters(text)
    doc = nlp(clean_text)
    subject_ls, verb_ls, object_ls = create_svo_lists(doc, print_lists=print_lists)
    
    graph_tup_ls = []
    dedup_tup_ls = []
    clean_tup_ls = []
    
    for subj in subject_ls: 
        for obj in object_ls:
            
            dist_ls = []
            
            for v in verb_ls:
                
                # Assemble a list of distances between each object and each verb
                dist_ls.append(abs(obj[1] - v[1]))
                
            # Get the index of the verb with the smallest distance to the object 
            # and return that verb
            index_min = min(range(len(dist_ls)), key=dist_ls.__getitem__)
            
            # Remve stop words from subjects and object.  Note that we do this a bit
            # later down in the process to allow for proper sentence recognition.

            no_sw_subj = remove_stop_words_and_punct(subj[0])
            no_sw_obj = remove_stop_words_and_punct(obj[0])
            
            # Add entries to the graph iff neither subject nor object is blank
            if no_sw_subj and no_sw_obj:
                tup = (no_sw_subj, verb_ls[index_min][0], no_sw_obj)
                graph_tup_ls.append(tup)
        
        #clean_tup_ls = remove_dates(graph_tup_ls)
    
    dedup_tup_ls = remove_duplicates(graph_tup_ls, 2)
    clean_tup_ls = remove_dates(dedup_tup_ls)
    
    return clean_tup_ls

## Helper Functions

In [6]:
def get_obj_properties(tup_ls):
    
    init_obj_tup_ls = []
    
    for tup in tup_ls:

        try:
            text, node_label_ls, url = query_google(tup[2], api_key, limit=1)
            new_tup = (tup[0], tup[1], tup[2], text[0], node_label_ls[0], url[0])
        except:
            new_tup = (tup[0], tup[1], tup[2], [], [], [])
        
        init_obj_tup_ls.append(new_tup)
        
    return init_obj_tup_ls


def add_layer(tup_ls):

    svo_tup_ls = []
    
    for tup in tup_ls:
        
        if tup[3]:
            svo_tup = create_svo_triples(tup[3])
            svo_tup_ls.extend(svo_tup)
        else:
            continue
    
    return get_obj_properties(svo_tup_ls)
        

def subj_equals_obj(tup_ls):
    
    new_tup_ls = []
    
    for tup in tup_ls:
        if tup[0] != tup[2]:
            new_tup_ls.append((tup[0], tup[1], tup[2], tup[3], tup[4], tup[5]))
            
    return new_tup_ls


def check_for_string_labels(tup_ls):
    # This is for an edge case where the object does not get fully populated
    # resulting in the node labels being assigned to string instead of list.
    # This may not be strictly necessary and the lines using it are commnted out
    # below.  Run this function if you come across this case.
    
    clean_tup_ls = []
    
    for el in tup_ls:
        if isinstance(el[2], list):
            clean_tup_ls.append(el)
            
    return clean_tup_ls


def create_word_vectors(tup_ls):

    new_tup_ls = []
    
    for tup in tup_ls:
        if tup[3]:
            doc = nlp(tup[3])
            new_tup = (tup[0], tup[1], tup[2], tup[3], tup[4], tup[5], doc.vector)
        else:
            new_tup = (tup[0], tup[1], tup[2], tup[3], tup[4], tup[5], np.random.uniform(low=-1.0, high=1.0, size=(300,)))
        new_tup_ls.append(new_tup)
        
    return new_tup_ls

## Create the node and edge lists to populate the graph with the below helper functions

In [7]:
def dedup(tup_ls):
    
    visited = set()
    output_ls = []
    
    for tup in tup_ls:
        if not tup[0] in visited:
            visited.add(tup[0])
            output_ls.append((tup[0], tup[1], tup[2], tup[3], tup[4]))
            
    return output_ls


def convert_vec_to_ls(tup_ls):
    
    vec_to_ls_tup = []
    
    for el in tup_ls:
        vec_ls = [float(v) for v in el[4]]
        tup = (el[0], el[1], el[2], el[3], vec_ls)
        vec_to_ls_tup.append(tup)
        
    return vec_to_ls_tup


def add_nodes(tup_ls):   

    keys = ['name', 'description', 'node_labels', 'url', 'word_vec']
    merge_nodes(graph.auto(), tup_ls, ('Node', 'name'), keys=keys)
    print('Number of nodes in graph: ', graph.nodes.match('Node').count())
    
    return

In [8]:
def add_edges(edge_ls):
    
    edge_dc = {} 
    
    # Group tuple by verb
    # Result: {verb1: [(sub1, v1, obj1), (sub2, v2, obj2), ...],
    #          verb2: [(sub3, v3, obj3), (sub4, v4, obj4), ...]}
    
    for tup in edge_ls: 
        if tup[1] in edge_dc: 
            edge_dc[tup[1]].append((tup[0], tup[1], tup[2])) 
        else: 
            edge_dc[tup[1]] = [(tup[0], tup[1], tup[2])] 
    
    for edge_labels, tup_ls in tqdm(edge_dc.items()):   # k=edge labels, v = list of tuples
        
        tx = graph.begin()
        
        for el in tup_ls:
            source_node = nodes_matcher.match(name=el[0]).first()
            target_node = nodes_matcher.match(name=el[2]).first()
            if not source_node:
                source_node = Node('Node', name=el[0])
                tx.create(source_node)
            if not target_node:
                try:
                    target_node = Node('Node', name=el[2], node_labels=el[4], url=el[5], word_vec=el[6])
                    tx.create(target_node)
                except:
                    continue
            try:
                rel = Relationship(source_node, edge_labels, target_node)
            except:
                continue
            tx.create(rel)
        tx.commit()
    
    return

## Read File

In [9]:
f = open('./data/unsup_data.txt', 'r')
text = f.read()

In [10]:
clean_text = remove_special_characters(text)
doc = nlp(clean_text)

In [11]:
text = clean_text.replace('. ', '\n')

In [12]:
words = [token.text
         for token in doc
         if not token.is_stop and not token.is_punct]

In [13]:
word_freq = Counter(words)

In [14]:
rows = text.split('\n')

In [15]:
rows[:10]

['It is made quite clear that the supplier shall be required to fulfill and implement the duties, obligations and responsibilities laid down in the pertinent applicable standard [IATF 16949:2016]',
 "Purpose of the document Particular attention is drawn to the following duties and obligations: Purpose of the document The supplier shall determine the customer's requirements in accordance with section 4.3.2 of [IATF 16949:2016] and satisfy them in an effort to improve customer satisfaction",
 'Purpose of the document Among other things, this means that the supplier shall determine requirements that have not been specified by the client but are necessary for the specified or intended use in accordance with section 8.2.2 of [IATF 16949:2016], and shall be obliged to define the properties of the product that are essential to ensure that it can be used safely for its intended purpose in accordance with section 8.2.3 of [IATF 16949:2016]',
 'Purpose of the document Furthermore, the supplier s

## Parse text and build graph

In [16]:
def text_to_kg(text:str, failed_texts:list, print_lists:bool):
    try:
        initial_tup_ls = create_svo_triples(text, print_lists=print_lists)
        init_obj_tup_ls = get_obj_properties(initial_tup_ls)
        new_layer_ls = add_layer(init_obj_tup_ls)
        starter_edge_ls = init_obj_tup_ls + new_layer_ls
        edge_ls = subj_equals_obj(starter_edge_ls)
        #clean_edge_ls = check_for_string_labels(edge_ls)
        #clean_edge_ls[0:3]
        clean_edge_ls = edge_ls

        edges_word_vec_ls = create_word_vectors(edge_ls)

        orig_node_tup_ls = [(edge_ls[0][0], '', ['Subject'], '', np.random.uniform(low=-1.0, high=1.0, size=(300,)))]
        obj_node_tup_ls = [(tup[2], tup[3], tup[4], tup[5], tup[6]) for tup in edges_word_vec_ls]
        full_node_tup_ls = orig_node_tup_ls + obj_node_tup_ls
        dedup_node_tup_ls = dedup(full_node_tup_ls)

        node_tup_ls = convert_vec_to_ls(dedup_node_tup_ls)

        add_nodes(node_tup_ls)
        add_edges(edges_word_vec_ls)
    except:
        failed_texts.append(text)

## Connect to neo4j

In [17]:
graph = Graph("bolt://localhost:7999", name="neo4j", password="secret")
nodes_matcher = NodeMatcher(graph)

In [ ]:
%%time
failed_texts = []
for t in rows:
    text_to_kg(t, failed_texts, print_lists=False)

Number of nodes in graph:  4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.10it/s]


Number of nodes in graph:  13


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.36it/s]


Number of nodes in graph:  21


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29.74it/s]


Number of nodes in graph:  604


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 26.85it/s]


Number of nodes in graph:  606


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.82it/s]


In [ ]:
len(failed_texts)

In [ ]:
%%time
failed_texts = []
for t in failed_texts:
    text_to_kg(t, [], print_lists=True)

## ============================================

## Start Building

In [ ]:
text = 'Particular attention is drawn to the following duties and obligations: Purpose of the document'

In [ ]:
text_to_kg(text, failed_texts, [], print_lists=False)

In [ ]:
%%time
initial_tup_ls = create_svo_triples(text, print_lists=False)

In [ ]:
initial_tup_ls

In [ ]:
%%time
init_obj_tup_ls = get_obj_properties(initial_tup_ls)
print(init_obj_tup_ls)
new_layer_ls = add_layer(init_obj_tup_ls)
print(new_layer_ls)
starter_edge_ls = init_obj_tup_ls + new_layer_ls
edge_ls = subj_equals_obj(starter_edge_ls)
#clean_edge_ls = check_for_string_labels(edge_ls)
#clean_edge_ls[0:3]
clean_edge_ls = edge_ls

In [ ]:
edges_word_vec_ls = create_word_vectors(edge_ls)

## Creating some lists of tuples representing the node and edge lists

In [ ]:
orig_node_tup_ls = [(edge_ls[0][0], '', ['Subject'], '', np.random.uniform(low=-1.0, high=1.0, size=(300,)))]
obj_node_tup_ls = [(tup[2], tup[3], tup[4], tup[5], tup[6]) for tup in edges_word_vec_ls]
full_node_tup_ls = orig_node_tup_ls + obj_node_tup_ls
dedup_node_tup_ls = dedup(full_node_tup_ls)

In [ ]:
len(full_node_tup_ls), len(dedup_node_tup_ls)

In [ ]:
full_node_tup_ls

## Create the node list that will be used to populate the graph

In [ ]:
node_tup_ls = convert_vec_to_ls(dedup_node_tup_ls)

In [ ]:
graph = Graph("bolt://localhost:7999", name="neo4j", password="secret")
nodes_matcher = NodeMatcher(graph)

In [ ]:
%%time
add_nodes(node_tup_ls)

In [ ]:
%%time
add_edges(edges_word_vec_ls)

## Usage

In [ ]:
def get_word_vec_similarity(node1, node2, node_ls):
    
    node1_vec = [tup[4] for tup in node_ls if tup[0] == node1]
    node2_vec = [tup[4] for tup in node_ls if tup[0] == node2]
    
    return cosine_similarity(node1_vec, node2_vec)

In [ ]:
cs = get_word_vec_similarity('company', 'standard', dedup_node_tup_ls)
print(cs)

## Read PDF

In [ ]:
!pip install PyPDF2

import PyPDF2

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader("./data/test.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"

In [ ]:
len(reader.pages)

In [ ]:
text = reader.pages[20].extract_text()